In [1]:
import pandas as pd
import gensim
import numpy as np
from gensim.models import word2vec
import preprocessing

In [2]:
#Importazione del file relativo al post di stack già processati 

data = pd.read_csv('DB/Preprocessed_data.csv')

data

,id,title,tags,score,post_corpus,questions,processed_title
0,742,Class views in Django,django|views|oop,0.009946,class views django view points function proble...,Class views in Django view points to a functi...,class views django
1,2933,Create a directly-executable cross-platform GU...,user-interface|deployment|tkinter|release-mana...,0.009108,create directlyexecutable crossplatform gui ap...,Create a directly-executable cross-platform GU...,create directlyexecutable crossplatform gui ap...
2,19339,Transpose/Unzip Function (inverse of zip)?,list|matrix|transpose,0.027115,transposeunzip function inverse zip list tuple...,Transpose/Unzip Function (inverse of zip)? I h...,transposeunzip function inverse zip
3,21961,Date/time conversion using time.mktime seems w...,datetime,-0.000942,datetime conversion using timemktime seems wro...,Date/time conversion using timemktime seems wr...,datetime conversion using timemktime seems wrong
4,36139,How to sort a list of strings?,string|sorting,0.029628,sort list strings best way creating alphabetic...,How to sort a list of strings? What is the bes...,sort list strings
...,...,...,...,...,...,...,...
75113,69059271,Pandas conditional formatting not displaying b...,pandas|finance|pandas-datareader|pandas-styles,-0.000523,pandas conditional formatting displaying backg...,Pandas conditional formatting not displaying b...,pandas conditional formatting displaying backg...
75114,69059275,split pandas frame into multiple frames based ...,pandas,-0.000523,split pandas frame multiple frames based timed...,split pandas frame into multiple frames based ...,split pandas frame multiple frames based timed...
75115,69059605,How to check if a column of DataFrame contain ...,pandas|dataframe|types,-0.000942,check column dataframe contain float type exam...,How to check if a column of DataFrame contain ...,check column dataframe contain float type
75116,69059907,How to increase/change a value in a dataset wi...,pandas|for-loop,-0.001779,increasechange value dataset condition python ...,How to increase/change a value in a dataset wi...,increasechange value dataset condition python


In [3]:
'''
Per poter fare il training word embeddings
In questa sezione importiamo l'intero corpo del DB dalla colonna unica che contiene: titolo, body e risposta
'''

documents = [_text.split() for _text in np.array(data.post_corpus)]

In [4]:
#data[1]

In [5]:
#Pre Train Word Embeddings

word_2_vec_model = gensim.models.word2vec.Word2Vec(vector_size = 300, window = 7, min_count = 10, workers = 8 )

word_2_vec_model.build_vocab(documents)
words = list(word_2_vec_model.wv.index_to_key)
sizeVocab = len(words)

words
print('Size vocab:', sizeVocab)

#Train Word Embeddings
word_2_vec_model.train(documents, total_examples = len(documents), epochs = 32 )

#Salvataggio

word_2_vec_model.save('DB/word2vec_trained.bin')

Size vocab: 12850


In [6]:
word_2_vec_model = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained.bin')



print(word_2_vec_model.wv.most_similar("indexing"))
#print('-'*100)

[('slicing', 0.5753238797187805), ('slice', 0.48981744050979614), ('indices', 0.46909618377685547), ('broadcasting', 0.4588119387626648), ('slices', 0.4387252628803253), ('indexer', 0.4359417259693146), ('index', 0.4346660077571869), ('broadcasted', 0.4203093647956848), ('mask', 0.3822118937969208), ('loc', 0.3805985748767853)]


In [7]:
word_2_vec_model2 = gensim.models.word2vec.Word2Vec.load('DB/word2vec_trained2.bin')
print(word_2_vec_model2.wv.most_similar("lambda"))

[('function', 0.4221939444541931), ('applymap', 0.34007009863853455), ('apply', 0.3390945792198181), ('dfapply', 0.33658337593078613), ('func', 0.3347214162349701), ('lambdify', 0.32197797298431396), ('analytical', 0.3216312527656555), ('agg', 0.3094872236251831), ('numba', 0.3072204291820526), ('helper', 0.29920855164527893)]
